In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
plt.style.use('ggplot')

# Data Collection

Trong phần này, chúng ta sẽ cần thu thập dữ liệu giúp cho ta có cái nhìn tổng quát về thị trường chứng khoán của nước Mỹ. Dựa trên danh sách 100 công ty có vốn hóa cao nhất, ta có thể phần nào hiểu được những ngành nào đang được chú ý, xu hướng giao dịch trên từng ngành,... để từ đó đi vào chi tiết cổ phiếu của từng công ty trong các năm gần đây.  

## 1. Lấy dữ liệu bằng cách kết hợp Selenium và Pandas DataReader

In [18]:
opts = webdriver.ChromeOptions()
opts.headless = True
browser = webdriver.Chrome(options=opts)

In [19]:
browser.get('https://www.tradingview.com/markets/stocks-usa/market-movers-large-cap')
raw_data = []
for i in range(100):
    element = browser.find_element(By.XPATH, '//*[@id="js-category-content"]/div/div/div[2]/div[2]/div/div[2]/div[2]/div/div/div/table/tbody/tr[{i}]'.format(i=i+1))
    raw_data.append(element.text)

In [20]:
raw_data

['AAPL\nApple Inc\nD\n2.351TUSD 147.81USD −0.34% −0.50USD\nNeutral\n65.446M 9.674B 24.30 6.14USD 164K Electronic Technology',
 'MSFT\nMicrosoft Corp.\nD\n1.901TUSD 255.02USD 0.13% 0.33USD\nBuy\n21.528M 5.49B 27.45 9.33USD 221K Technology Services',
 'GOOG\nAlphabet Inc (Google) Class C\nD\n1.303TUSD 100.83USD −0.44% −0.45USD\nBuy\n18.821M 1.898B 20.38 5.09USD 156.5K Technology Services',
 'AMZN\nAmazon.com, Inc.\nD\n960.282BUSD 94.13USD −1.43% −1.37USD\nSell\n72.496M 6.824B 87.70 1.11USD 1.608M Retail Trade',
 'BRK.A\nBerkshire Hathaway Inc.\nD\n696.418BUSD 477402.98USD 0.07% 317.98USD\nBuy\n2.794K 1.334B — −1194.00USD 372K Finance',
 'TSLA\nTesla, Inc.\nD\n615.32BUSD 194.86USD 0.08% 0.16USD\nNeutral\n73.645M 14.35B 60.16 3.60USD 99.29K Consumer Durables',
 'UNH\nUnitedHealth Group Incorporated\nD\n500.961BUSD 536.16USD −0.14% −0.75USD\nBuy\n2.346M 1.258B 26.29 20.71USD 350K Health Services',
 'JNJ\nJohnson & Johnson\nD\n467.679BUSD 178.88USD 0.08% 0.14USD\nBuy\n5.824M 1.042B 24.88 7.2

In [21]:
browser.quit()

In [22]:
test = raw_data[0].split('\n')
test

['AAPL',
 'Apple Inc',
 'D',
 '2.351TUSD 147.81USD −0.34% −0.50USD',
 'Neutral',
 '65.446M 9.674B 24.30 6.14USD 164K Electronic Technology']

In [23]:
tickerSymbol = []
tickerCompanyName = []

for each_data in raw_data:
    resolveElement = each_data.split('\n')
    tickerSymbol.append(resolveElement[0])
    tickerCompanyName.append(resolveElement[1])
d = {"Symbol": tickerSymbol, "CompanyName": tickerCompanyName}
symbol_data = pd.DataFrame(data=d)
symbol_data

,Symbol,CompanyName
0,AAPL,Apple Inc
1,MSFT,Microsoft Corp.
2,GOOG,Alphabet Inc (Google) Class C
3,AMZN,"Amazon.com, Inc."
4,BRK.A,Berkshire Hathaway Inc.
...,...,...
95,VRTX,Vertex Pharmaceuticals Incorporated
96,EOG,"EOG Resources, Inc."
97,BKNG,Booking Holdings Inc. Common Stock
98,PGR,Progressive Corporation (The)


In [24]:
symbol_data.to_csv("symbol.csv", index=False)

In [25]:
market_data = pd.read_html('https://www.tradingview.com/markets/stocks-usa/market-movers-large-cap/')[0]
market_data

,Ticker,Market cap,Price,Chg % 1D,Chg 1D,Technical Rating 1D,Vol 1D,Volume * Price 1D,P/E(TTM),EPS(TTM),Employees,Sector
0,AAPLApple Inc,2.351TUSD,147.81USD,−0.34%,−0.50USD,Neutral,65.446M,9.674B,24.30,6.14USD,164K,Electronic Technology
1,MSFTMicrosoft Corp.,1.901TUSD,255.02USD,0.13%,0.33USD,Buy,21.528M,5.49B,27.45,9.33USD,221K,Technology Services
2,GOOGAlphabet Inc (Google) Class C,1.303TUSD,100.83USD,−0.44%,−0.45USD,Buy,18.821M,1.898B,20.38,5.09USD,156.5K,Technology Services
3,"AMZNAmazon.com, Inc.",960.282BUSD,94.13USD,−1.43%,−1.37USD,Sell,72.496M,6.824B,87.70,1.11USD,1.608M,Retail Trade
4,BRK.ABerkshire Hathaway Inc.,696.418BUSD,477402.98USD,0.07%,317.98USD,Buy,2.794K,1.334B,—,−1194.00USD,372K,Finance
...,...,...,...,...,...,...,...,...,...,...,...,...
95,VRTXVertex Pharmaceuticals Incorporated,82.493BUSD,321.37USD,0.19%,0.61USD,Buy,1.095M,351.862M,25.32,12.81USD,3.9K,Health Technology
96,"EOGEOG Resources, Inc.",81.63BUSD,138.97USD,−1.12%,−1.57USD,Neutral,3.933M,546.506M,11.04,12.82USD,2.8K,Energy Minerals
97,BKNGBooking Holdings Inc. Common Stock,80.893BUSD,2085.44USD,1.22%,25.15USD,Buy,270.497K,564.105M,33.83,61.20USD,20.3K,Consumer Services
98,PGRProgressive Corporation (The),77.528BUSD,132.51USD,0.95%,1.25USD,Strong Buy,1.253M,166.066M,92.89,1.42USD,49.077K,Finance


In [26]:
market_data.to_csv("market.csv", index=False)

## 2. Dùng API yfinance để thu thập dữ liệu về giá cổ phiếu thuộc 10 công ty có vốn hóa lớn nhất 

In [27]:
tickers = tickerSymbol[:10]
tickers

['AAPL', 'MSFT', 'GOOG', 'AMZN', 'BRK.A', 'TSLA', 'UNH', 'JNJ', 'XOM', 'V']

Để có thể lấy được dữ liệu từ yfinance về giá của `BRK.A`, ta cần sửa lại thành `BRK-A` 

In [28]:
for i in range(len(tickers)):
    if (tickers[i] == 'BRK.A'):
        tickers[i] = 'BRK-A'

tickers

['AAPL', 'MSFT', 'GOOG', 'AMZN', 'BRK-A', 'TSLA', 'UNH', 'JNJ', 'XOM', 'V']

In [29]:
end = datetime.now()
start = datetime(end.year - 4, end.month, end.day)

price_data = []
for ticker in tickers:
    price_data.append(yf.download(ticker, start, end))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [30]:
for i in range(len(tickers)):
    price_data[i]['Ticker'] = tickers[i]

price_data[0]

,Open,High,Low,Close,Adj Close,Volume,Ticker
Date,,,,,,,
2018-12-03,46.115002,46.235001,45.302502,46.205002,44.595947,163210000,AAPL
2018-12-04,45.237499,45.597500,44.067501,44.172501,42.634228,165377200,AAPL
2018-12-06,42.939999,43.695000,42.605000,43.680000,42.158878,172393600,AAPL
2018-12-07,43.372501,43.622501,42.075001,42.122501,40.655613,169126400,AAPL
2018-12-10,41.250000,42.522499,40.832500,42.400002,40.923454,248104000,AAPL
...,...,...,...,...,...,...,...
2022-11-28,145.139999,146.639999,143.380005,144.220001,144.220001,69246000,AAPL
2022-11-29,144.289993,144.809998,140.350006,141.169998,141.169998,83763800,AAPL
2022-11-30,141.399994,148.720001,140.550003,148.029999,148.029999,111380900,AAPL


In [31]:
price_data = pd.concat(price_data)
price_data

,Open,High,Low,Close,Adj Close,Volume,Ticker
Date,,,,,,,
2018-12-03,46.115002,46.235001,45.302502,46.205002,44.595947,163210000,AAPL
2018-12-04,45.237499,45.597500,44.067501,44.172501,42.634228,165377200,AAPL
2018-12-06,42.939999,43.695000,42.605000,43.680000,42.158878,172393600,AAPL
2018-12-07,43.372501,43.622501,42.075001,42.122501,40.655613,169126400,AAPL
2018-12-10,41.250000,42.522499,40.832500,42.400002,40.923454,248104000,AAPL
...,...,...,...,...,...,...,...
2022-11-28,212.259995,214.470001,210.649994,211.259995,211.259995,10229400,V
2022-11-29,209.350006,210.149994,207.350006,209.059998,209.059998,8986900,V
2022-11-30,208.220001,217.000000,208.009995,217.000000,217.000000,13918100,V


In [32]:
price_data.to_csv('price.csv')